In [0]:
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT REPLACE(SPLIT(current_user(), '@')[0], '.', '_');

In [0]:
USE IDENTIFIER("fhir_workshop." || schema_use);

In [0]:
SELECT current_catalog(), current_schema();

In [0]:
DECLARE OR REPLACE VARIABLE patient_keys STRING;

SET VAR patient_keys = (
  SELECT 
    array_join(collect_list(DISTINCT resource), "', '")
  FROM 
    fhir_resource_schemas
  WHERE 
    resourceType = 'Patient'
);

select patient_keys;

In [0]:
DROP TABLE IF EXISTS Patient;

In [0]:
DECLARE OR REPLACE VARIABLE patient_resource_stmnt STRING;

SET VARIABLE patient_resource_stmnt = "
CREATE OR REFRESH STREAMING TABLE Patient 
COMMENT 'Parsed FHIR Patient Records'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  *
FROM (
  SELECT
    bundle_uuid
    ,fullUrl
    ,key
    ,value
  FROM 
    STREAM(fhir_resources)
  WHERE 
    resourceType = 'Patient')
  PIVOT (
    first(value) FOR key IN ('" || patient_keys || "')
  );
";

SELECT patient_resource_stmnt;


In [0]:
EXECUTE IMMEDIATE patient_resource_stmnt;

In [0]:
select * from patient limit 10;